# Video Processing for BT Lab
Functions to check if run is good
<br>
<br>
![UofC logo](./pictures/uofc_logo-black.jpg)

In [28]:
#import libraries
import os
import re
import json
from pathlib import Path
import numpy as np
import pandas as pd
import traceback
from timeit import default_timer as timer
import cv2
from typing import List, Optional, Tuple

# plot
from matplotlib import pyplot as plt
import matplotlib.patches as patches

In [2]:
# faster r-cnn
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn

In [3]:
# import sk libraries
from sklearn.model_selection import train_test_split

In [5]:
# make sure to update path
user_drive = input("Enter user drive: ").upper()
video_path = f"{user_drive}:/Christian/DI_centre_structured"
input(f"Is this the right directory - {video_path}?")

''

In [6]:
VIDEO_CHARACTERISTICS = {
    "With Blankets" : "WB",
    "B" : "WB",
    "Without Blankets" : "WOB",
    "WOB": "WOB",
    "3 Meters" : "3m",
    "2 Meters" : "2m",
    "Hold Breath" : "HB",
    "Hold Breathe" : "HB",
    "H" : "HB",
    "Relaxed" : "rel",
    "R": "rel",
}

# for testing
FRAME_LIMIT = 100

# limit for storage
MINIMUM_FREE_SPACE_GB = 75

In [7]:
# local dirs
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"
log_dir = repo_dir + "/records/logs"

In [12]:
def get_video_frame_paths(local_path: str, level: str) -> List[str]:
    """
    Constructs and returns paths related to video frames.

    Parameters:
    local_path (str): The local file path of the video.
    level (str): The detail level for the frames.

    Returns:
    List[str]: A list containing the folder path for frames and the video folder path.
    """
    video_folder, video_filename_with_ext = os.path.split(local_path)
    video_filename = os.path.splitext(video_filename_with_ext)[0]
    folder_path = os.path.join(video_folder, f"frames_{video_filename}_{level}")
    return [folder_path, video_folder]

In [13]:
def resample_frames(old_fps: int, new_fps: int, start: int) -> List[int]:
    """
    Resamples the number of frames from an old frame rate to a new frame rate. 
    This function can handle both upsampling and downsampling.

    Parameters:
    old_fps (int): The original frames per second.
    new_fps (int): The new frames per second to resample to.
    start (int): The starting frame index.

    Returns:
    List[int]: A list of frame indices after resampling.

    Raises:
    ValueError: If old_fps or new_fps are non-positive integers.
    """
    if old_fps <= 0 or new_fps <= 0:
        raise ValueError("old_fps and new_fps must be positive integers.")

    original_frame_indices = np.arange(start, old_fps, dtype=int)
    interpolated_frame_positions = np.linspace(start, old_fps - 1, new_fps)
    nearest_frame_indices = np.round(interpolated_frame_positions).astype(int)
    resampled_frame_indices = np.take(original_frame_indices, nearest_frame_indices, mode='wrap')

    return resampled_frame_indices.tolist()

In [15]:
def create_patient_video_id(patient_data: dict, video_count: int) -> str:
    """
    Creates a unique ID for a patient video scenario.

    Parameters:
    patient_data (dict): Dictionary containing patient information.
    video_count (int): The count of videos for the patient.

    Returns:
    str: A unique ID string for the video.
    """
    alias = patient_data.get("alias", "Unknown")
    blanket = VIDEO_CHARACTERISTICS.get(patient_data.get("blanket", ""), "?")
    distance = VIDEO_CHARACTERISTICS.get(patient_data.get("distance", "").title(), "?")
    breathing = VIDEO_CHARACTERISTICS.get(patient_data.get("breathing", ""), "?")

    return f"{alias}_{video_count}-{distance}-{blanket}-{breathing}"

In [40]:
def folder_exists(folder_path: str) -> bool:
    """Checks if the specified folder exists."""
    return os.path.exists(folder_path)

def extract_video_metadata(video_path: str) -> Tuple[int, int]:
    """Extracts metadata from the video file."""
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        raise RuntimeError(f"Failed to open video file {video_path}")

    try:
        vid_fps = int(video.get(cv2.CAP_PROP_FPS))
        total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    finally:
        video.release()

    return vid_fps, total_frames

def calculate_expected_frame_count(vid_fps: int, total_frames: int, new_fps: int) -> int:
    """Calculates the expected number of frames."""
    vid_duration = total_frames // vid_fps
    return new_fps * vid_duration

def count_frames_in_folder(folder_path: str) -> int:
    """Counts the number of frames (files) in the given folder."""
    return len(os.listdir(folder_path))

def run_folder_check(video_path: str, save_folder: str, new_fps: int) -> Optional[str]:
    """
    Checks if the specified folder has the correct number of frames extracted from a video.
    Returns the folder path if the frame count does not match.
    """
    try:
        vid_fps, total_frames = extract_video_metadata(video_path)
        expected_frame_count = calculate_expected_frame_count(vid_fps, total_frames, new_fps)
        actual_frame_count = count_frames_in_folder(save_folder)

        if actual_frame_count != expected_frame_count:
            print(f"{save_folder} - Incorrect frame count. Found {actual_frame_count}, expected {expected_frame_count}.")
            return save_folder
    except RuntimeError as e:
        print(str(e))
        return None

    return None


In [45]:
# driver code for checking empty folders
def validate_run(all_patients:dict, level: str, new_fps:int) -> list:
    visited_folders = {}    
    rerun_folders = []
    frames_folders = []

    for json_index, patient_info in all_patients.items():
        try:
            video_path = patient_info["local path"]
            old_fps = int(patient_info["old fps"])

            frames_folder, video_folder = get_video_frame_paths(video_path, level)
            visited_folders[video_folder] = visited_folders.get(video_folder, 0) + 1
            frames_folders.append(frames_folder)

            frames_to_pick = resample_frames(old_fps, new_fps, 1)

            if len(frames_to_pick) != new_fps:
                raise ValueError("Number of frames to pick is not equal to new fps")

            # checks if a folder has the correct number of frames
            rerun_folder = run_folder_check(video_path, frames_folder, new_fps)
            
            if rerun_folder != None:
                print("Need to rerun folder: ", rerun_folder)
                rerun_folders.append(rerun_folder)

        except Exception as e:
            traceback.print_exc()
            print(f'''{type(e)}: {e} for video {patient_info["filename"]}''')

    return [rerun_folders, frames_folders]

In [56]:
# export to json

def export_to_json(filename: str, all_patient_info: dict) -> None:
    patient_json = json.dumps(all_patient_info, indent=2)

    with open(filename, "w") as json_data:
        json_data.write(patient_json)

In [9]:
def load_json(json_dir: str, filename: str) -> dict:
    # Remove leading slash if present in filename
    if filename.startswith("/"):
        filename = filename[1:]

    full_path = os.path.join(json_dir, filename)

    try:
        with open(full_path, "r") as json_data:
            return json.load(json_data)
    except FileNotFoundError:
        print(f"Error: The file {full_path} does not exist.")
        return {}
    except json.JSONDecodeError:
        print(f"Error: The file {full_path} is not a valid JSON.")
        return {} 

# Run tests

In [21]:
""" local vars"""

rgb_fps = {
    "lower_bound": 10,
    "upper_bound": 20
}

thermal_fps = {
    "lower_bound": 5,
    "upper_bound": 10
}

In [17]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "/rgb_complete.json")
metadata_thermal = load_json(json_dir, "/thermal_complete.json")

In [46]:
""" check videos (rgb) """
all_rgb_folders = []
all_rgb_rerun_folders = []

for level, new_fps in rgb_fps.items():
    print(f"\nAdjusting FPS to {new_fps}\n" + "="*50)
    rerun_folders, visited_folders = validate_run(metadata_rgb, level, new_fps)
    all_rgb_rerun_folders.append(rerun_folders)
    all_rgb_folders.append(visited_folders)


Adjusting FPS to 10
C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun\2 Meters\With Blankets\Relaxed\frames_Arun2_lower_bound - Incorrect frame count. Found 17, expected 20.
Need to rerun folder:  C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun\2 Meters\With Blankets\Relaxed\frames_Arun2_lower_bound
C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun\2 Meters\Without Blankets\Hold Breath\frames_short_lower_bound - Incorrect frame count. Found 17, expected 290.
Need to rerun folder:  C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun\2 Meters\Without Blankets\Hold Breath\frames_short_lower_bound
C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun\2 Meters\Without Blankets\Relaxed\frames_relax 2meter short_lower_bound - Incorrect frame count. Found 17, expected 640.
Need to rerun folder:  C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun\2 Meters\Without Blankets\Relaxed\frames_relax 2meter short_lower_bound

Adjusting FPS t

In [60]:
""" check videos (thermal) """
# all_thermal_folders = []
# all_thermal_rerun_folders = []

# for level, new_fps in thermal_fps.items():
#     print(f"\nAdjusting FPS to {new_fps}\n" + "="*50)
#     rerun_folders, visited_folders = check_if_empty_folders(metadata_thermal, level, new_fps, user_drive)
#     all_thermal_folders.append(visited_folders)
#     all_thermal_rerun_folders.append(rerun_folders)

' check videos (thermal) '

# Crop Image

In [77]:
def detect_objects(img, model, threshold=0.5):
    # Convert the image to RGB and then to a tensor
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    transform = T.Compose([T.ToTensor()])
    img_tensor = transform(img_rgb)

    # Put the model in evaluation mode and apply to the image
    model.eval()
    with torch.no_grad():
        prediction = model([img_tensor])

    # Collect bounding boxes
    bboxes = []
    for element in range(len(prediction[0]['boxes'])):
        if prediction[0]['scores'][element] > threshold:
            box = prediction[0]['boxes'][element].cpu().numpy()
            label = prediction[0]['labels'][element].cpu().numpy()
            
            if label == 1:  # COCO Dataset label for 'person'
                bboxes.append(box)

    return bboxes

In [69]:
# Load a pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)

c:\Users\chris\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\chris\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [74]:
all_rgb_lower_bound, all_rgb_upper_bound = all_rgb_folders
all_rgb_lower_bound

['C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames_Arun2_lower_bound',
 'C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Relaxed/frames_Arun2_lower_bound',
 'C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/Without Blankets/Hold Breath/frames_short_lower_bound',
 'C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/Without Blankets/Relaxed/frames_relax 2meter short_lower_bound']

In [81]:
for rgb_folder in all_rgb_lower_bound:
    first_image = rgb_folder + "/" + os.listdir(rgb_folder)[0]
    print(first_image)

    img = cv2.imread(first_image)

    if img is None:
        print("Error: Could not read the image.")
    else:
        bounding_boxes = detect_objects(img, model)
        print("Detected bounding boxes:", bounding_boxes)
    break

C:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Hold Breath/frames_Arun2_lower_bound/15_1-2m-WB-HB_0.png
Detected bounding boxes: [array([  82.82521,  316.22778,  785.2172 , 1029.9728 ], dtype=float32)]


# Getting labels

In [61]:
# gets labels based on breathing

def get_label_breathing(visited_folders: list[str]) -> [list, list]:
    all_frames_path = []
    all_labels = []

    hold_breath_pattern = r"/Hold Breath/|/Hold Breathe/|/H/"
    relaxed_pattern = r"/Relaxed/|/R/"

    for visited_folder in visited_folders:
        if re.search(relaxed_pattern, visited_folder):
            all_labels.append(0) # 0 for relaxed pattern
        elif re.search(hold_breath_pattern, visited_folder):
            all_labels.append(1) # 1 for hold breath pattern
        else:
            print(f"Warning: No matching breathing pattern for {visited_folder}")
            continue

        all_frames_path.append(visited_folder)

    return [all_frames_path, all_labels]

In [62]:
# get labels for rgb

# all_rgb_frames_path = []
# all_rgb_labels = []
# save_data = {}

# for fps_bound in all_rgb_folders:
#     tmp_path_list, tmp_labels_list = get_label_breathing(fps_bound)
#     all_rgb_frames_path += tmp_path_list
#     all_rgb_labels += tmp_labels_list

# save_data["frames_path"] = all_rgb_frames_path
# save_data["labels_path"] = all_rgb_labels

In [63]:
# save test train split in json data (rgb)
# save_rgb_filename = json_dir + "/training_test_split/rgb_labels.json"
# export_to_json(save_rgb_filename, save_data)

# Splitting the data into training and testing

In [64]:
# data splitting

def split_data(all_videos: list[str], all_labels: list[int]):
    train_videos, test_videos, train_labels, test_labels = train_test_split(
        all_videos, all_labels, test_size=0.2, random_state=42)
    return train_videos, test_videos, train_labels, test_labels


In [65]:
# split data (rgb)
# train_rgb_frames, test_rgb_frames, train_rgb_labels, test_rgb_labels = train_test_split(
#     all_rgb_frames_path, all_rgb_labels, test_size=0.2, random_state=42
# )